<a href="https://colab.research.google.com/github/AmadoMaria/hands-on-supercomputing-with-parallel-computing/blob/master/hackathon_2022.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<div align="center"><h1>HPC / SENAI / Hackathon (2022.2)<br>
Brute Force Algorithm </h1></div>


M. Amado$^1$, F. Lisboa$^1$

$^1$ Department of Computer Engenier – University SENAI CIMATEC, Salvador, Bahia, Brazil 

## Introdução

<div align="justify">
<p>
As técnicas de paralelismo compreendem em aplicação de estratégias baseadas na utilização de processamento paralelo manipulando diferentes recursos computacionais. Alguma dessas técnicas compreendem a utilização de bibliotecas paralelas como <code>OpenMP</code>, <code>MPI</code> e <code>CUDA</code>. Cada uma dessas bibliotecas consiste na manipulação de recursos computacionais diferentes, podendo ser utilizadas de forma híbrida, a fim da obtenção de máximo desempenho. No OpenMP e CUDA manipulamos <i>Threads</i>, enquanto no MPI, <i>Processos</i>, sendo de real relevância os impactos destas unidades de processamento frente aos recursos computacionais alocados. 

A seguir será apresentado um código sequencial para a quebra de senha de até 20 caracteres utilizando um algoritmo de <i>Força Bruta</i>. O objetivo básico será inserir técnicas de paralelismo ao código, tal que serão considerados alguns itens nas aplicações finais:

</p>


* `Análise dos Custos Computacionais das Aplicações Sequênciais e Paralelas`
    + Profilling CPU (gprof)
    + Profiling GPU (nsys)
* `Estudo das Estruturas Algorítmicas das Aplicações Paralelas`
    + Modelos Algorítmicos Aplicados
    + Características da inserção da API
* `Análise de Desempenho`
     + Experimentação de Parâmetros Ótimos (Melhores valores de Processos, Threads e Grid Computacional)
     + Indices de eficiência (Speedup)


</div>

<div style="justify">
<p>
Para a comparação entre as equipes foi fornecida a senha: <code><i>_Hacka1</i></code>.
</p>
</div>

## Código Sequencial 

In [ ]:
%%writefile bruteForce.c

#include <stdio.h>
#include <stdlib.h>
#include <string.h>
#include <time.h>
#include <math.h>

//97 to 122 use only lowercase letters
//65 to 90 use only capital letters
//48 to 57 use only numbers

#define START_CHAR 48
#define END_CHAR 122
#define MAXIMUM_PASSWORD 20

long long my_pow(long long x, int y)
{
  long long res = 1;
  if (y==0)
    return res;
  else
    return x * my_pow(x, y-1);
}

void bruteForce(char *pass) 
{
  char force[MAXIMUM_PASSWORD];
  int palavra[MAXIMUM_PASSWORD];
  int pass_b26[MAXIMUM_PASSWORD];
    
  long long int j;
  long long int pass_decimal = 0;
  int base = END_CHAR - START_CHAR + 2;

  int size = strlen(pass);

  for(int i = 0; i < MAXIMUM_PASSWORD; i++)
    force[i] = '\0';

  printf("Try to broke the password: %s\n", pass);

  for(int i = 0; i < size; i++)
    pass_b26[i] = (int) pass[i] - START_CHAR + 1; 

  for(int i = size - 1; i > -1; i--)
    pass_decimal += (long long int) pass_b26[i] * my_pow(base, i);

  long long int max = my_pow(base, size);
  char s[MAXIMUM_PASSWORD];

  for(j = 0; j < max; j++){
    if(j == pass_decimal){
      printf("Found password!\n");
      int index = 0;

      printf("Password in decimal base: %lli\n", j);
      while(j > 0){
        s[index++] = START_CHAR + j%base-1;
        j /= base;
      }
      s[index] = '\0';
      printf("Found password: %s\n", s);
      break;
    }
  }

}

int main(int argc, char **argv) 
{
  char password[MAXIMUM_PASSWORD];
  strcpy(password, argv[1]);
  time_t t1, t2;
  double dif;

  time (&t1);
    bruteForce(password);
  time (&t2);

  dif = difftime (t2, t1);

  printf("\n%1.2f seconds\n", dif);

  return 0;
}

Writing bruteForce.c


In [ ]:
!chmod 777 bruteForce.c

In [ ]:
!gcc bruteForce.c -o bruteForce -std=c99 -O3

In [ ]:
!./bruteForce zzzzzz

Try to broke the password: zzzzzz
Found password!
Password in decimal base: 387420488
Found password: zzzzzz

0.00 seconds


### Análise dos Custos Computacionais da Aplicação Sequencial

<div align="justify">
<p>
A análise dos custos computacionais foi realizada utilizando o <i>GNU Profiler</i>, sendo possível afirmar que a função de <i>bruteForce</i> representa a maior parte do tempo de execução do programa. É possível verificar essa análise a partir dos comandos a seguir.


</p>
</div>

In [ ]:
!gcc bruteForce.c -o bruteForce -std=c99 -O3 -pg

In [ ]:
!./bruteForce _Hacka1

Try to broke the password: senhatez
Found password!
Password in decimal base: 274193963128
Found password: senhatez

94.00 seconds


In [ ]:
!gprof -b bruteForce gmon.out

Flat profile:

Each sample counts as 0.01 seconds.
  %   cumulative   self              self     total           
 time   seconds   seconds    calls  Ts/call  Ts/call  name    
100.17     93.59    93.59                             bruteForce


---
## Estudo das Estruturas Algorítmicas das Aplicações Paralelas

### Multicore (OpenMP)

<div align="justify">
<p>
A paralelização do código de força bruta foi realizada utilizando o OpenMP, o qual aplica o paradigma de memória compartilhada. Dessa forma, dentro da função <i>bruteForce</i>, o <i>for</i> responsável por encontrar o valor decimal da senha enviada como parâmetro, e assim, realizar sua quebra, foi paralelizado. Esse bloco de código foi escolhido, uma vez que representa um <i>loop</i> que vai de zero até o valor máximo possível para a senha, ou seja, 27 elevado ao número de caracteres.
</p>
</div>

In [18]:
%%writefile bruteForce-omp.c

#include <stdio.h>
#include <stdlib.h>
#include <string.h>
#include <time.h>
#include <math.h>
#include <omp.h>

//97 to 122 use only lowercase letters
//65 to 90 use only capital letters
//48 to 57 use only numbers

#define START_CHAR 48
#define END_CHAR 122
#define MAXIMUM_PASSWORD 20

long long my_pow(long long x, int y)
{
  long long res = 1;
  if (y==0)
    return res;
  else
    return x * my_pow(x, y-1);
}

void bruteForce(char *pass) 
{
  time_t t1, t2;
  double dif;
  time (&t1);

  char force[MAXIMUM_PASSWORD];
  int palavra[MAXIMUM_PASSWORD];
  int pass_b26[MAXIMUM_PASSWORD];
    
  long long int j;
  long long int pass_decimal = 0;
  int base = END_CHAR - START_CHAR + 2;

  int size = strlen(pass);

  for(int i = 0; i < MAXIMUM_PASSWORD; i++)
    force[i] = '\0';

  printf("Try to broke the password: %s\n", pass);

  for(int i = 0; i < size; i++)
    pass_b26[i] = (int) pass[i] - START_CHAR + 1; 

  for(int i = size - 1; i > -1; i--)
    pass_decimal += (long long int) pass_b26[i] * my_pow(base, i);

  long long int max = my_pow(base, size);
  char s[MAXIMUM_PASSWORD];
  
  // Esse loop é o responsável pela maior parte do custo computacional
  #pragma omp parallel for private(j)
  for(j = 0; j < max; j++){
    if(j == pass_decimal){
      printf("Found password!\n");
      int index = 0;

      printf("Password in decimal base: %lli\n", j);
      while(j > 0){
        s[index++] = START_CHAR + j%base-1;
        j /= base;
      }
      s[index] = '\0';
      printf("Found password: %s\n", s);
      time (&t2);
      dif = difftime (t2, t1);

      printf("\n%1.2f seconds\n", dif);
      exit(0); //com esse exit não exibe o tempo
    }  
  }
}

int main(int argc, char **argv) 
{
  char password[MAXIMUM_PASSWORD];
  strcpy(password, argv[1]);
  
  bruteForce(password);
  
  return 0;
}

Writing bruteForce-omp.c


In [19]:
!chmod 777 bruteForce-omp.c

In [20]:
!gcc bruteForce-omp.c -o bruteForce-omp -fopenmp -std=c99 -O3

In [ ]:
!OMP_NUM_THREADS=30 ./bruteForce-omp zzzzzzzz

Try to broke the password: zzzzzzzz


<div align="justify">
<p>
Como o código OpenMP recebe como parâmetro o número de threads utilizadas na execução, foi necessário identificar a quantidade ótima de threads. Portanto, a quebra da senha <code>_Hacka1</code> foi realizada diversas vezes, com números de threads correspondentes a potências de 2. A partir do código sequêncial, os speedups correspondentes foram cálculados, conforme a Figura 2. Assim, foi possível notar que a melhor execução do programa ocorre com 32 threads, que apresenta um speedup de aproximadamente 4.
<p>
</div>

<div align="center">
<img src="https://github.com/AmadoMaria/hands-on-supercomputing-with-parallel-computing/blob/master/Hackathon/results__Hacka1/omp_time.png?raw=true" alt="Tempo de execução por threads no OpenMP" />

<i><b>Figura 1</b>: Tempo de execução por threads no OpenMP</i>

</div>


<div align="center">
<img src="https://github.com/AmadoMaria/hands-on-supercomputing-with-parallel-computing/blob/master/Hackathon/results__Hacka1/speed_up_omp.png?raw=true" alt="Speedups por threads no OpenMP" />

<i><b>Figura 2</b>: Speedups por threads no OpenMP</i>

</div>

### Multiprocessor (MPI)

<div align="justify">
<p>
A paralelização efetuada no código MPI foi pautada na execução da função <i>bruteForce</i> por múltiplos processos, em que cada um deles ficou encarregado de executar um intervalo de valors específico na execução do mesmo <i>loop</i> paralelizado em OpenMP. Assim, cada processo percorreu um intervalo que inicia na variável <i>lower_bound</i> e vai até a variável <i>upper_bound</i>, calculadas dinamicamente de acordo com o número de processos em execução. 

Para essa implementação, a função <i>bruteForce</i> recebeu como parâmetros, além da senha, o <i>rank</i> do processo e o número total de processos.
</p>
</div>

In [ ]:
%%writefile bruteForce-mpi.c
#include <stdio.h>
#include <stdlib.h>
#include <string.h>
#include <time.h>
#include <math.h>
#include <mpi.h>
#include <omp.h>

// 97 to 122 use only lowercase letters
// 65 to 90 use only capital letters
// 48 to 57 use only numbers

#define START_CHAR 48
#define END_CHAR 122
#define MAXIMUM_PASSWORD 20

long long my_pow(long long x, int y)
{
    long long res = 1;
    if (y == 0)
        return res;
    else
        return x * my_pow(x, y - 1);
}

void bruteForce(char *pass, int rank, int numberProcess)
{
    char force[MAXIMUM_PASSWORD];
    int palavra[MAXIMUM_PASSWORD];
    int pass_b26[MAXIMUM_PASSWORD];

    long long int j;
    long long int pass_decimal = 0;
    int base = END_CHAR - START_CHAR + 2;

    int size = strlen(pass);

    for (int i = 0; i < MAXIMUM_PASSWORD; i++)
        force[i] = '\0';

    for (int i = 0; i < size; i++)
        pass_b26[i] = (int)pass[i] - START_CHAR + 1;

    for (int i = size - 1; i > -1; i--)
        pass_decimal += (long long int)pass_b26[i] * my_pow(base, i);

    long long int max = my_pow(base, size);
    char s[MAXIMUM_PASSWORD];

    long long partition = ceil(max / numberProcess);
    long long int rest = max % numberProcess;
    
    if (rest != 0){
      partition += rest;
    }

    long long lower_bound = rank * partition;
    long long upper_bound = (rank + 1) * partition;
    for (j = lower_bound; j < upper_bound; j++)
    {
        if (j == pass_decimal)
        {
            printf("Found password!\n");
            int index = 0;

            printf("Password in decimal base: %lli\n", j);
            while (j > 0)
            {
                s[index++] = START_CHAR + j % base - 1;
                j /= base;
            }
            s[index] = '\0';
            printf("Found password: %s\n", s);
            return;
        }
    }
    MPI_Finalize();
    exit(0);
}

int main(int argc, char **argv)
{
    char password[MAXIMUM_PASSWORD];
    strcpy(password, argv[1]);
    time_t t1, t2;
    double dif;

    int numberProcess, rank;
    MPI_Init(&argc, &argv);
    MPI_Comm_rank(MPI_COMM_WORLD, &rank);
    MPI_Comm_size(MPI_COMM_WORLD, &numberProcess);
    MPI_Status status;

    if (rank == 0)
        printf("Try to broke the password: %s\n", password);

    time(&t1);
    bruteForce(password, rank, numberProcess);
    time(&t2);

    dif = difftime(t2, t1);

    printf("\n%1.2f seconds\n", dif);
    MPI_Finalize();
    return 0;
}

Writing bruteForce-mpi.c


In [ ]:
!chmod 777 bruteForce-mpi.c

In [ ]:
!mpicc bruteForce-mpi.c -o bruteForce-mpi -fopenmp -std=c99 -O3

In [ ]:
!mpirun --allow-run-as-root -np 30 ./bruteForce-mpi zzzzzzz

Try to broke the password: zzzzzzz
Found password!
Password in decimal base: 10460353202
Found password: zzzzzzz

4.00 seconds


<div align="justify">
<p>
Para identificar o melhor número de processos para a execução do código MPI, realizou-se um processo semelhante ao OpenMP, sendo que o número de processos foi incrementado de acordo com os múltiplos de 2. Assim, com 30 processos o maior speedup foi alcançado, sendo considerado o parâmetro ótimo para execução.
</p>
</div>

<div align="center">

<img src="https://github.com/AmadoMaria/hands-on-supercomputing-with-parallel-computing/blob/master/Hackathon/results__Hacka1/mpi_time.png?raw=true" alt="Tempo de execução por processos no MPI" />

<i><b>Figura 3</b>: Tempo de execução por processos no MPI</i>

</div>




<div align="center">
<img src="https://github.com/AmadoMaria/hands-on-supercomputing-with-parallel-computing/blob/master/Hackathon/results__Hacka1/speed_up_mpi.png?raw=true" alt="Speedups por processos no MPI" />

<i><b>Figura 4</b>: Speedups por processos no MPI</i>

</div>




### Multiprocessor + Multicore (MPI + OpenMP)

<div align="justify">
<p>
O código MPI, onde cada processo é responsável por uma parte do problema, foi incrementado com a paralelização do <i>loop</i> principal, da mesma forma que foi realizada no código OpenMP. Assim, além de cada processo percorrer um intervalo, sua porção do problema conta com otimização proporcionada pelo uso de threads.

</p>
</div>

In [ ]:
%%writefile bruteForce-mpi+omp.c

#include <stdio.h>
#include <stdlib.h>
#include <string.h>
#include <time.h>
#include <math.h>
#include <mpi.h>

//97 to 122 use only lowercase letters
//65 to 90 use only capital letters
//48 to 57 use only numbers

#define START_CHAR 97
#define END_CHAR 122
#define MAXIMUM_PASSWORD 20

long long my_pow(long long x, int y)
{
  long long res = 1;
  if (y==0)
    return res;
  else
    return x * my_pow(x, y-1);
}

void bruteForce(char *pass, int rank, int numberProcess) 
{
  time_t t1, t2;
  double dif;
  time (&t1);
  
  char force[MAXIMUM_PASSWORD];
  int palavra[MAXIMUM_PASSWORD];
  int pass_b26[MAXIMUM_PASSWORD];
    
  long long int j;
  long long int pass_decimal = 0;
  int base = END_CHAR - START_CHAR + 2;

  int size = strlen(pass);

  for(int i = 0; i < MAXIMUM_PASSWORD; i++)
    force[i] = '\0';

  for(int i = 0; i < size; i++)
    pass_b26[i] = (int) pass[i] - START_CHAR + 1; 

  for(int i = size - 1; i > -1; i--)
    pass_decimal += (long long int) pass_b26[i] * my_pow(base, i);

  long long int max = my_pow(base, size);
  char s[MAXIMUM_PASSWORD];

  // O problema é dividido de acordo com o número de processos
  long long partition = ceil(max / numberProcess);
  long long int rest = max % numberProcess;
  // verificação se contém resto e se max < numberProcess
  if (rest != 0){
    partition += rest;
  }

  // Definindo o intervado de responsabilidade do processo
  long long lower_bound = rank * partition;
  long long upper_bound = (rank + 1) * partition;

  // Paralelização do loop com openmp
  #pragma omp parallel for private(j)
  for(j = lower_bound; j < upper_bound; j++){
      if(j == pass_decimal){
        printf("Found password!\n");
        int index = 0;

        printf("Password in decimal base: %lli\n", j);
        while(j > 0){
          s[index++] = START_CHAR + j%base-1;
          j /= base;
        }
        s[index] = '\0';
        printf("Found password: %s\n", s);
        time (&t2);
        dif = difftime (t2, t1);
        printf("\n%1.2f seconds\n", dif);

        // Como a senha foi encotrada os processoas são finalizados
        MPI_Abort(MPI_COMM_WORLD, MPI_SUCCESS);
      }
  }
}

int main(int argc, char **argv) 
{
  char password[MAXIMUM_PASSWORD];
  strcpy(password, argv[1]);

  int numberProcess, rank;
  MPI_Init(&argc, &argv);
  MPI_Comm_rank(MPI_COMM_WORLD, &rank);
  MPI_Comm_size(MPI_COMM_WORLD, &numberProcess);
  MPI_Status status;
    
  if(rank==0) // Essa mensagem só é executada uma vez
    printf("Try to broke the password: %s\n", password);
    
  
  bruteForce(password, rank, numberProcess);

  return 0;
}

Overwriting bruteForce-mpi+omp.c


In [ ]:
!chmod 777 bruteForce-mpi+omp.c

In [ ]:
!mpicc bruteForce-mpi+omp.c -o bruteForce-mpi+omp -fopenmp -std=c99 -O3

In [ ]:
!OMP_NUM_THREADS=4 mpirun -x MXM_LOG_LEVEL=error -np 22 --allow-run-as-root ./bruteForce-mpi+omp zzzzzzzzzz

Try to broke the password: zzzzzzzzzz


<div align="center">
<img src="https://github.com/AmadoMaria/hands-on-supercomputing-with-parallel-computing/blob/master/Hackathon/results__Hacka1/openmpi_time.png?raw=true" alt="Tempo de execução por threads e processos no OpenMPI" />

<i><b>Figura 5</b>: Tempo de execução por threads e processos no OpenMPI</i>

</div>

<br />
<br />

<div align="justify">
<p>
Percebe-se que há uma variação entre a quantidade de processos e threads durante as execuções. Entretanto, visualiza-se uma linearidade na curva de tempo para 16 threads entre o intervalo de processos executados.
</p>
</div>


<div align="center">
<img src="https://github.com/AmadoMaria/hands-on-supercomputing-with-parallel-computing/blob/master/Hackathon/results__Hacka1/speed_up_openmpi.png?raw=true" alt="Speedups por threads e processos no OpenMPI" />


<i><b>Figura 6</b>: Speedups por threads e processos no OpenMPI</i>

</div>

<br />
<br />

<div align="justify">
<p>
Comparando redução e aumento entre os speedups, é nítido que a execução com 4 threads e 28 processos pode ser considerada o melhor valor obtido em relação aos outros, já que logo nas quantidades seguintes de processos, esse cálculo é reduzido, de maneira linear no início. 

Observando as curvas, também percebe-se um destaque quanto aos maiores speedups obtidos com 4, 8 e 32 threads, para 22, 26 e 30 processos.

</p>
</div>


### GPU (CUDA)

<div align="justify">
<p>
O código CUDA, utilizando GPU, se baseia em: 


*   Transformação da *bruteForce* em função kernel;
*   Utilização do conceito *host*-*device*;
*   Uso de Grid computacional;
*   Uso de *stride* para a distribuição do tamanho da senha de acordo com a quantidade de *threads* máxima disponível por bloco na GPU;
*   Cada *thread* acessa um dos valores no *device*;
*   Verificação de erros com a utilização da GPU.


</p>
</div>

In [ ]:
%%writefile bruteForce-cuda.cu

#include <stdio.h>
#include <stdlib.h>
#include <string.h>
#include <time.h>
#include <math.h>
#include <assert.h>

//97 to 122 use only lowercase letters
//65 to 90 use only capital letters
//48 to 57 use only numbers

#define START_CHAR 48
#define END_CHAR 122
#define MAXIMUM_PASSWORD 20

inline cudaError_t checkCuda(cudaError_t result)
{
    if (result != cudaSuccess)
    {
        fprintf(stderr, "CUDA Runtime Error: %s\n", cudaGetErrorString(result));
        assert(result == cudaSuccess);
    }
    return result;
}

__device__ long long my_pow(long long x, int y)
{
  long long res = 1;
  if (y==0)
    return res;
  else
    return x * my_pow(x, y-1);
}


__global__ void bruteForce(char *pass, long long size) 
{
  char force[MAXIMUM_PASSWORD];
  int palavra[MAXIMUM_PASSWORD];
  int pass_b26[MAXIMUM_PASSWORD];
    
  long long int pass_decimal = 0;
  int base = END_CHAR - START_CHAR + 2;

  for(int i = 0; i < MAXIMUM_PASSWORD; i++)
    force[i] = '\0';

  for(int i = 0; i < size; i++)
    pass_b26[i] = (int) pass[i] - START_CHAR + 1; 

  for(int i = size - 1; i > -1; i--){
    pass_decimal += (long long int) pass_b26[i] * my_pow(base, i);
  }
    
  long long max = my_pow(base, size);
  
  long long start = threadIdx.x + blockIdx.x * blockDim.x;
  for (long long idx = start; idx < max; idx += gridDim.x * blockDim.x) {
    if (idx > pass_decimal) return;
    if(idx == pass_decimal){
      int index = 0;
      char s[MAXIMUM_PASSWORD];

      printf("Password in decimal base: %lli\n", idx);
      while((idx) > 0){
        s[index++] = START_CHAR + idx%base-1;
        idx /= base;
      }
      s[index] = '\0';
      printf("Found password: %s\n", s);
      return;
    }
  }
}

int main(int argc, char **argv) 
{
  char *password;
  time_t t1, t2;
  double dif;

  checkCuda(cudaMallocManaged(&password, sizeof(char) * MAXIMUM_PASSWORD));

  cudaError_t syncErr, asyncErr;

  strcpy(password, argv[1]);
  int size = strlen(password);

  int deviceId, numberOfSMs;
  cudaGetDevice(&deviceId);
  cudaDeviceGetAttribute(&numberOfSMs, cudaDevAttrMultiProcessorCount, deviceId);
  int number_of_blocks = numberOfSMs * 32;
  int threads_per_block = 1024;

  printf("Try to broke the password: %s\n", password);

  time (&t1);
  bruteForce<<<number_of_blocks, threads_per_block>>>(password, size);
  syncErr = cudaGetLastError();
  asyncErr = cudaDeviceSynchronize();
  time (&t2);

  if (syncErr != cudaSuccess)
      printf("Error: %s\n", cudaGetErrorString(syncErr));
  if (asyncErr != cudaSuccess)
      printf("Error: %s\n", cudaGetErrorString(asyncErr));
  
  dif = difftime (t2, t1);

  printf("\n%1.2f seconds\n", dif);
  cudaFree(password);

  return 0;
}

Writing bruteForce-cuda.cu


In [ ]:
!chmod 777 bruteForce-cuda.cu

In [ ]:
!nvcc bruteForce-cuda.cu -o bruteForce-cuda -x cu 

bruteForce-cuda.cu(39): warning: variable "force" was set but never used

bruteForce-cuda.cu(40): warning: variable "palavra" was declared but never referenced



In [ ]:
!./bruteForce-cuda zzzzzzzz

Try to broke the password: zzzzzzzzz


## Análise de Desempenho

### Parâmetros Ótimos de Execução

1. OpenMP = 32 Threads
2. MPI = 1 nó + 30 Processos
3. MPI + OpenMP = 1 nó + 22 Processos + 4 Threads
4. CUDA = G1D B1DT1D (80 * 32, 1024)

### Tempo de execução em segundos das aplicações

<div align="justify">
<p>
A fim de comparar os diferentes tempos de execução do código de força bruta, no supercomputador, a senha escolhida foi <code>_Hacka1</code>. Dessa forma, a paralelização com o CUDA apresentou o menor tempo de execução, de 1 segundo, seguida pela implementação com MPI, que durou 16 segundos. Em comparação, o código sequencial realizou a quebra da senha em 136 segundos, enquanto os códigos OpenMP e OpenMPI levaram, respectivamente, 35 e 46 segundos em execução.
</p>
</div>

<div align="center">

<img src="https://github.com/AmadoMaria/hands-on-supercomputing-with-parallel-computing/blob/master/Hackathon/results__Hacka1/Time.png?raw=true" alt="Melhor tempo de execução em cada estrutura" />


<i><b>Figura 7</b>: Melhor tempo de execução em cada estrutura</i>

</div>





<div align="justify">
<p>
A Tabela 1, contém o tempo de execução do algoritmo sequencial e das paralelizações, para diferentes senhas, utilizando os parâmetros obtidos para os melhores speedups na senha de teste (<code>_Hacka1</code>). Vale ressaltar que nessas execuções o intervalo de caracteres foi reduzido - com o início modificado de 97 para 48, visando apenas letras minúsculas. Logo, utilizando o mesmo que foi executado para a senha de teste, o tempo de execução entre as estruturas seria muito maior, pois abrangeria também caracteres especiais e letras maiúsculas, aumentando, consequentemente, muito mais o tamnho do loop principal do algoritmo. 

<br />
<br />

Configuração da execução das estruturas de paralelização:

*   OpenMP: 32 threads
*   MPI: 30 processos
*   OpenMPI: 4 threads e 22 processos
*   Cuda: 32 blocos

<br />

Observando esses dados, é possível notar que a paralelização com o CUDA ainda se mostrou a mais eficiente. Contudo, para a cadeia com 9 caracteres, o OpenMP obeteve um tempo de execução menor do que o MPI, o oposto do que ocorreu com a senha teste <code>_Hacka1</code>.

</p>
</div>

<div align="center">

**Tabela 1**: Tempos de execução em segundos obtidos para cada senha em cada estrutura

|  Senha    | Sequencial |  OpenMP    | MPI     | Híbrido       | CUDA
| --------- |  :------:  |  :------: | :------:  | :------:    | :------:
| (5z) zzzzz | 0 | 0 | 0 | 0 | 0
| (6z) zzzzzz | 0 | 0 | 0 | 0 | 0
| (7z) zzzzzzz |2        | 0           | 0     |  0             | 0  
| (8z) zzzzzzzz  | 75      | 6           | 5     |10               | 0  
| (9z) zzzzzzzzz   | 1998     | 144           | 152     | 269               | 8

</div>

### Speedup

<div align="justify">
<p>
Como a paralelização com o CUDA foi a mais rápida, consequentemente, obteve o melhor speedup, com uma execução 136 vezes mais rápida do que que o código sequencial. No gráfico apresentado na Figura 8, também é possível verificar que o segundo maior speedup foi de 8x, com o MPI. Já o OpenMP e OpenMPI, apresentaram os piores resultados em comparação aos outros, entretanto, seus speedups ainda conseguiram ser maiores do que 2x, com speedups de 3x e 4x, respectivamente.
<p>
</div>

<div align="center">
<img src="https://github.com/AmadoMaria/hands-on-supercomputing-with-parallel-computing/blob/master/Hackathon/results__Hacka1/Speedups.png?raw=true" alt="Melhores speedups encontrados em cada estrutura" />

<i><b>Figura 8</b>: Melhor speedup encontrado em cada estrutura.</i>

</div>







A Tabela 2, contruída com base na tabela dos tempos de execução, mostra os speedups obtidos para cadeias de diferentes tamanhos utilizando os parâmetros para os melhores speedups em cada estrutura para a senha de teste (<code>_Hacka1</code>).

<div align="center">

**Tabela 2**: Speedups obtidos em cada estrutura para cada senha.

|  Senha    |  OpenMP    | MPI     | Híbrido       | CUDA
| --------- | :------:   | :------:  | :------:     | :------:
| (5z) zzzzz | -          |  -       | -             |  -
| (6z) zzzzzz | -          |  -       | -              |  -
| (7z) zzzzzzz   |  $$\infty $$          |   $$\infty $$       |  $$\infty $$              |   $$\infty $$
| (8z) zzzzzzzz        | 12,5x           |  15x       | 7,5x              |  $$\infty $$  
| (9z) zzzzzzzzz        | 13,9x           |  13,2x       | 7,4x              | 249,8x 

</div>

## Conclusões

<div align="justify">
<p>
Durante o hackathon, foi possível comparar a implementação e desempenho de diferentes técnicas de paralelização, com a finalidade de otimizar um algoritmo de força bruta. A fim comparar os tempos de execução dos códigos, a senha "_Hacka1", foi a escolhida para realização das quebras. Assim, a implementação que se mostrou mais eficiente foi a do código CUDA, com um speedup de 136x. Nessa estrutura, utilizamos um "<i>grid-stride loop</i>" para possibilitar a quebra de senhas de qualquer tamanho, independentemente do número de threads e blocos configurados. Já as paralelizações com OpenMP e OpenMPI foram as mais lentas com speedups de 4x e 3x, enquanto o código MPI obteve um speedup de até 8x. Dessa forma, percebemos que o CUDA foi a ferramenta que proporcionou o melhor uso dos recursos do supercomputador, evidenciando os benefícios da computação em GPU.

</p>
</div>


## Referências Biliográficas

G. Coulouris, J. Dollimore, T. Kindberg, G.Blair. Distributed Systems: Concepts and Design, Fifth Edition, Pearson, 2011.

S.Tanenbaum, M. Steen, Distributed Systems: Principles and Paradigms, Second Edition, Pearson, 2006.

David A. Patterson and John L. Hennessy. Computer Orga- nization and Design: The Hardware/Software Interface. Morgan Kaufmann, 5th Edition, 2013.

An Introduction to Parallel Programming by Peter S. Pache- co. Morgan Kauffman.

W. C. Barbosa, An introduction to distributed algorithms, MIT Press, 1997. N. Lynch, Distributed Algorithms, Mit Press, 1996 e Introduction to Distributed Algorithms, Gerard Tel, Cabri- bridge U. Press, 1994.
